In [6]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.agents import Tool, AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda

# 환경 변수
load_dotenv()
OPEN_API_KEY = os.getenv("OPENAI_API_KEY")


# 기존 도구 함수들
def add_two(x: str) -> str:
    try:
        return str(float(x) + 2)
    except Exception:
        raise ValueError("숫자만 입력해 주세요.")


def population_lookup(city: str) -> str:
    data = {
        "서울": "9,733,509명 (2025년 기준)",
        "부산": "3,400,000명 (2025년 기준)",
    }
    result = data.get(city.strip().lower())
    if result:
        return result
    else:
        raise ValueError(f"{city}의 인구 정보를 찾을 수 없습니다.")


# 도구 및 LLM 설정
tools = [
    Tool(name="AddTwo", func=add_two, description="숫자 입력 시 2를 더합니다."),
    Tool(
        name="PopulationLookup",
        func=population_lookup,
        description="도시 이름 입력 시 인구 정보를 반환합니다.",
    ),
]

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0, openai_api_key=OPEN_API_KEY, verbose=True)

# 프롬프트 설정
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 도움이 되는 어시스턴트입니다. 주어진 도구를 사용해서 질문에 답하세요.",
        ),
        ("human", "{input}"),
        MessagesPlaceholder("agent_scratchpad"),
    ]
)

# Agent 생성
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


# Fallback 함수
def llm_fallback(inputs):
    user_input = inputs.get("input", "")
    response = llm.invoke(f"다음 질문에 답해주세요: {user_input}")
    return {
        "output": f"🤖 도구에서 정보를 찾지 못해 일반 지식으로 답변합니다:\n{response.content}",
        "intermediate_steps": [],
    }


# Fallback이 적용된 Agent
smart_agent = agent_executor.with_fallbacks([RunnableLambda(llm_fallback)])

# 테스트
if __name__ == "__main__":
    # 도구에서 찾을 수 있는 정보
    print("=== 도구에서 찾을 수 있는 정보 ===")
    result1 = smart_agent.invoke({"input": "서울의 인구는?"})
    print(result1["output"])

    print("\n=== 도구에서 찾을 수 없는 정보 (LLM Fallback) ===")
    result2 = smart_agent.invoke({"input": "대구의 인구는?"})
    print(result2["output"])

=== 도구에서 찾을 수 있는 정보 ===


> Entering new AgentExecutor chain...

Invoking: `PopulationLookup` with `서울`


9,733,509명 (2025년 기준)서울의 인구는 9,733,509명입니다 (2025년 기준).

> Finished chain.
서울의 인구는 9,733,509명입니다 (2025년 기준).

=== 도구에서 찾을 수 없는 정보 (LLM Fallback) ===


> Entering new AgentExecutor chain...

Invoking: `PopulationLookup` with `대구`


🤖 도구에서 정보를 찾지 못해 일반 지식으로 답변합니다:
대구의 인구는 2023년 기준으로 약 240만 명 정도입니다. 하지만 인구는 시간이 지남에 따라 변동할 수 있으므로, 최신 정보를 확인하는 것이 좋습니다. 대구의 인구에 대한 정확한 수치는 대구광역시청이나 통계청의 공식 자료를 참고하시기 바랍니다.
